In [89]:
import os
import time
import sys
import re

import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains

In [90]:
PATH = 'C:/21_hf216/project/CrawlingData/'
df = pd.read_csv(PATH + 'isbn.csv')

print(df.shape)
df.head()

(770, 2)


,0,1
0,9772384289005,미스테리아 35호
1,9772384367000,악스트 Axt 2021.7.8 - no.037
2,9772508333003,릿터 Littor 2021.6.7 - 30호
3,9772733807003,"에픽 #04 - BELOVED, 2021.7.8.9"
4,9788901166223,월든 - 시민 불복종 수록


In [91]:
isbn_list = df.iloc[:,0]
isbn_list = isbn_list.apply(str)
isbn_list

0      9772384289005
1      9772384367000
2      9772508333003
3      9772733807003
4      9788901166223
           ...      
765    9791197120541
766    9791197182204
767    9791197381706
768    9791197459702
769    9791197475818
Name: 0, Length: 770, dtype: object

In [92]:
# 검색창에 isbn 입력하는 함수
def close_popup():
    css = '#divYes24SCMEvent > div.yPopBot'
    popup = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css)))
    btnClick1 = ActionChains(driver).move_to_element(popup).click()
    btnClick1.perform()
    print('팝업 닫기')
    time.sleep(1)

def search_isbn(isbn):
    css = '#query'
    search_box = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    # search_box = driver.find_element_by_css_selector(class_)
    search_box.send_keys(isbn)
    search_box.submit()
    if search_box:
        print(f'{isbn} 검색 완료')
    
    time.sleep(1)

# 검색 후 작가 클릭 하는 함수
def click_author():
    driver.refresh()
    css = '#schMid_wrap > div:nth-child(4) > div.goodsList.goodsList_list > table > tbody > tr:nth-child(1) > td.goods_infogrp > div.goods_info > a'
    # author = driver.find_element_by_css_selector(css)
    author = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css)))
    btnClick = ActionChains(driver).move_to_element(author).click()
    btnClick.perform()
    print('작가 버튼 클릭')
    time.sleep(1)

def click_more():
    driver.refresh()
    css_more = '#schMid_wrap > div.insideYes24.insideTp01.clearfix > div > dl > dd > a'
    more = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css_more)))
    btnClick2 = ActionChains(driver).move_to_element(more).click()
    btnClick2.perform()
    print('작가 더보기 버튼 클릭')
    time.sleep(1)

def get_info_text(author_infolist):
    driver.refresh()
    info_text = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, 'txt150'))).text
    # info_text = driver.find_element_by_class_name('txt150').text
    author_infolist.append([info_text])
    print('작가 정보 저장')
    time.sleep(1)
    return info_text


In [ ]:
chromedriver = 'C:/chromedriver'
driver = webdriver.Chrome(chromedriver)
driver.get('http://www.yes24.com/main/default.aspx')

In [ ]:
chromedriver = 'C:/chromedriver'
driver = webdriver.Chrome(chromedriver)
driver.get('http://www.yes24.com/main/default.aspx')
li = []
search_isbn(isbn_list[4])
click_author()
click_more()
get_info_text(li)
print(li)


In [93]:
author_infolist = []
error_list = []
chromedriver = 'C:/chromedriver'
driver = webdriver.Chrome(chromedriver)
driver.get('http://www.yes24.com/main/default.aspx')
close_popup()
iter = 0
for isbn in isbn_list:
    iter += 1
    driver.get('http://www.yes24.com/main/default.aspx')
    try:
        search_isbn(isbn)
        click_author()
        click_more()
        author_info = get_info_text(author_infolist)
    except Exception as e:
        print(e)
        author_infolist.append([np.nan])
        author_info = np.nan
        error_list.append([iter, isbn, e])
    with open('author_info2.csv', encoding='utf-8-sig', mode='a') as f:
        f.write(f'{author_info}\n')

    
        
        

팝업 닫기
9772384289005 검색 완료
작가 버튼 클릭
Message: 

9772384367000 검색 완료
작가 버튼 클릭
Message: 

9772508333003 검색 완료
작가 버튼 클릭
Message: 

9772733807003 검색 완료
작가 버튼 클릭
작가 더보기 버튼 클릭
작가 정보 저장
9788901166223 검색 완료
작가 버튼 클릭
작가 더보기 버튼 클릭
작가 정보 저장
9788901215907 검색 완료
작가 버튼 클릭
작가 더보기 버튼 클릭
작가 정보 저장
9788901248202 검색 완료
작가 버튼 클릭
작가 더보기 버튼 클릭
작가 정보 저장
9788901252278 검색 완료
작가 버튼 클릭
작가 더보기 버튼 클릭
작가 정보 저장
9788908060197 검색 완료
작가 버튼 클릭
Message: 

9788917230505 검색 완료
작가 버튼 클릭
작가 더보기 버튼 클릭
작가 정보 저장
9788925538297 검색 완료
작가 버튼 클릭
작가 더보기 버튼 클릭
작가 정보 저장
9788925554990 검색 완료
작가 버튼 클릭
작가 더보기 버튼 클릭
작가 정보 저장
9788925568782 검색 완료
작가 버튼 클릭
작가 더보기 버튼 클릭
작가 정보 저장
9788925588377 검색 완료
작가 버튼 클릭
작가 더보기 버튼 클릭
작가 정보 저장
9788925588643 검색 완료
작가 버튼 클릭
작가 더보기 버튼 클릭
작가 정보 저장
9788925588650 검색 완료
작가 버튼 클릭
작가 더보기 버튼 클릭
작가 정보 저장
9788925588711 검색 완료
작가 버튼 클릭
작가 더보기 버튼 클릭
작가 정보 저장
9788925588735 검색 완료
작가 버튼 클릭
작가 더보기 버튼 클릭
작가 정보 저장
9788925588759 검색 완료
작가 버튼 클릭
작가 더보기 버튼 클릭
작가 정보 저장
9788925588957 검색 완료
작가 버튼 클릭
작가 더보기 버튼 클릭
작가 정보 저장
9788925591711 검색 완

In [113]:
author_series = pd.Series()
for i in range(len(author_infolist)):
    author_series = author_series.append(pd.Series(author_infolist[i][0]))
author_series.reset_index(drop=True, inplace=True)
author_series

C:\ProgramData\Anaconda3\envs\tens_2g\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


0                                                    NaN
1                                                    NaN
2                                                    NaN
3      1983년 전남 순천에서 태어났다. 2014년 [현대문학] 신인추천에 단편소설 『그...
4      1817년 매사추세츠 주 콩코드에서 태어났다. 자신을 ‘신비주의자, 초절주의자, 자...
                             ...                        
765    어린이청소년문학 작가. 1962년 충북 청원군에서 나고 서울에서 자랐다. 유년기부터...
766    여러 도시와 해외를 전전하다 포항에서 도서출판 이팝을 운영 중이다. 생계형 기자였다...
767    진주에서 태어나 서울에서 자랐다. 어렸을 때, 골방에서 홀로 책을 읽던 시간의 후유...
768    1957년 도쿄에서 태어났다. 국제상과대학 상학부를 졸업한 뒤 조모신문(上毛新聞)에...
769    1985년 경기도 성남에서 태어나 부산에서 어린 시절을 보냈다. 주민등록증이 나왔을...
Length: 770, dtype: object

In [120]:
sub = pd.DataFrame(dict(isbn = isbn_list, author_info = author_series)).reset_index(drop=True)
sub['author_info'].fillna('작가 정보가 없습니다.', inplace=True)
sub.to_csv('author_info.csv', encoding='utf-8-sig', index=False)
sub

,isbn,author_info
0,9772384289005,작가 정보가 없습니다.
1,9772384367000,작가 정보가 없습니다.
2,9772508333003,작가 정보가 없습니다.
3,9772733807003,1983년 전남 순천에서 태어났다. 2014년 [현대문학] 신인추천에 단편소설 『그...
4,9788901166223,"1817년 매사추세츠 주 콩코드에서 태어났다. 자신을 ‘신비주의자, 초절주의자, 자..."
...,...,...
765,9791197120541,어린이청소년문학 작가. 1962년 충북 청원군에서 나고 서울에서 자랐다. 유년기부터...
766,9791197182204,여러 도시와 해외를 전전하다 포항에서 도서출판 이팝을 운영 중이다. 생계형 기자였다...
767,9791197381706,"진주에서 태어나 서울에서 자랐다. 어렸을 때, 골방에서 홀로 책을 읽던 시간의 후유..."
768,9791197459702,1957년 도쿄에서 태어났다. 국제상과대학 상학부를 졸업한 뒤 조모신문(上毛新聞)에...
